<a href="https://colab.research.google.com/github/HassanRiaz1992/Resume-Parser-in-Spacy-3/blob/main/Resume_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [54]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
!pip install pyresparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import pandas as pd

In [58]:
#cv_data = pd.read_pickle(r'/content/sample_data/Resume/train_data.pkl')

In [59]:
cv_data = json.load(open('/content/train_data.json'))

In [60]:
len(cv_data)

200

In [61]:
!python -m spacy init fill-config /content/sample_data/Resume/base_config.cfg config.cfg

2023-06-14 20:33:18.645972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [62]:
cv_data[3]

["Ijas Nizamuddin Associate Consultant - State Street  Irinchayam B.O, Kerala - Email me on Indeed: indeed.com/r/Ijas- Nizamuddin/6748d77f76f94eed  With close to 3 years of experience in IT industry, I have had excellent exposure to design, development and implementation of Client Server Applications in various domains such as Banking and Finance concepts. I have been involved in various software Development projects in Open System environment.  WORK EXPERIENCE  Associate Consultant  Oracle Corporation -  June 2011 to Present  State Street Global Advisors (SSgA) is the asset management business of State Street Corporation, one of the world's leading providers of financial services to institutional investors1, with a heritage dating back over two centuries. Backed by the strength and stability of the State Street organization, SSgA makes continual investments in asset management and client service platform, resulting in a client-focused, solutions-driven orientation .BrokerViews is the 

In [63]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en') #created an empty pipeline for english language
    db = DocBin() # created a doc bin object
      # iterate the dataset which we want to convert into spacy format
    for text, annot in tqdm(data):
        # convert the dataset of resume into text format
        doc = nlp.make_doc(text)
        # identify the annotation from data
        annot = annot['entities']
        # store the entities into an array
        ents = []
        # Number of characteristics
        entity_indices = []

        # Extracting information of entities from the data set in iteration
        #start, end and label are like 16, 36, 'Designation'
        for start, end, label in annot:
            # No Entity shall be skip from extraction
            skip_entity = False
            # if an entity character is already present in the entity_indices than we shall skip the entity and and cotinure the process
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue
            # entity indices are passed to the array of entity_indices and list of star and end maintained
            entity_indices = entity_indices + list(range(start, end))
             # we need to check that either annotated data is producing error or not
            try:
                span = doc.char_span(start, end,label=label,alignment_mode = 'strict')
            except:
                continue
              # if span has nothing than we need tocheck that at span has error
            if span is None:
                err_data = str([start, end]) + " " + str(text) + "\n "
                file.write(err_data)
            else:
                ents.append(span)
        # putting all the extracted entities in the in doc which later added to docbin
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
    return db

In [64]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)

In [65]:
 len(train),len(test)

(160, 40)

In [66]:
file = open('error.txt', 'w', encoding='utf-8')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 40/40 [00:00<00:00, 42.35it/s]


In [67]:

len(db.tokens)

40

In [68]:
#import subprocess
# Spacy command
#config_path = "/content/config.cfg"
#train_data_path = "./content/train_data.spacy"
#test_data_path = "./content/test_data.spacy"
#output_path = "./output"

#command = ["python", "-m", "spacy", "train", config_path, "--output", output_path, "--paths.train", train_data_path, "--paths.dev", test_data_path]
#subprocess.run(command)

CompletedProcess(args=['python', '-m', 'spacy', 'train', '/content/config.cfg', '--output', './output', '--paths.train', './content/train_data.spacy', '--paths.dev', './content/test_data.spacy'], returncode=1)

In [69]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-06-14 20:33:54.796463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-14 20:34:03,710] [INFO] Set up nlp object from config
[2023-06-14 20:34:03,728] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-14 20:34:03,731] [INFO] Created vocabulary
[2023-06-14 20:34:03,732] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Ro

In [72]:
NLP = spacy.load('/content/output/model-best')

In [73]:
import PyPDF2
# Load the language model
#nlp = spacy.load("en_core_web_sm")

# Open the PDF file
pdf_file = open("/content/Smith Resume.pdf", "rb")

# Read the PDF file
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Extract the text from the first page
page = pdf_reader.pages[0]
cv_text = page.extract_text ()
cv_text

'Michael Smith \nBI / Big Data/ Azure  \nManchester , UK- Email me on Indeed:  indeed.com/r/ falicent/140749dace5dc26f  \n \n10+ years of Experience in Designing, Development, Administration, Analysis, \nManagement inthe Business Intelligence Da ta warehousing, Client Server \nTechnologies, Web -based Applications, cloud solutions and Databases.  \nData warehouse: Data analysis, star/ snow flake schema data modeling and design \nspecific todata warehousing and business intelligence environment.  \nDatabase: Experience in database designing, scalability, back -up and recovery, \nwriting andoptimizing SQL code and Stored Procedures, creating functions, views, \ntriggers and indexes.  \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL \nAzure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure \ndata lake analytics(U -SQL). \nBig Data: Worked Azure data lake store/analytics for big data processing and Azure \ndata factoryto schedule U -SQ

In [74]:
doc = NLP(cv_text)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

NAME   =>   Michael Smith
LOCATION   =>   Manchester
EMAIL ADDRESS   =>   indeed.com/r/ falicent/140749dace5dc26f
YEARS OF EXPERIENCE   =>   10+
DESIGNATION   =>   Software Engineer
COMPANIES WORKED AT   =>   Microsoft
LOCATION   =>   Manchester
COMPANIES WORKED AT   =>   Microsoft
